In [3]:
project_list = [
    'commons-codec',
    # 'commons-net',
    'delight-nashorn-sandbox',
    'empire-db',
    'jimfs'
]
round_number = 6
seed_list = [
    0,
    2024,
    99999
]
parsed_path = 'controlled_parsed_data'
analyzed_path = 'controlled_analyzed_data'

## 1 Find out Flaky Mutants

In [ ]:
import numpy as np
import json
from pitest_log_parser import mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'
seed_num = len(seed_list)


def get_info(project, seed):
    path = f'{parsed_path}/{choice}/{project}_{seed}'
    with open(f'{path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict


for project in project_list:
    pref_set = set()
    for seed in seed_list:
        mutantId_mutantTuple_dict, mutantId_runtimeList_dict = get_info(project, seed)
        per_set = set()
        for mutant_id, runtime_list in mutantId_runtimeList_dict.items():
            if any(np.isnan(runtime) for runtime in runtime_list if isinstance(runtime, float)):
                continue
            per_set.add(tuple(mutantId_mutantTuple_dict[mutant_id][:5]))
        if len(pref_set) == 0:
            pref_set |= per_set
        else:
            pref_set &= per_set
    for seed in seed_list:
        mutantId_mutantTuple_dict, mutantId_runtimeList_dict = get_info(project, seed)
        non_flaky_list = []
        for mutant_id, mutant_tuple in mutantId_mutantTuple_dict.items():
            if tuple(mutant_tuple[:5]) in pref_set:
                non_flaky_list.append(mutant_id)
        # with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'w') as file:
        #     json.dump(non_flaky_list, file, indent=4)
    print(f'{project}: {len(non_flaky_list)}')

## 2 Calculate T-test, U-test

In [ ]:
import pandas as pd
import warnings
import json
from scipy.stats import ttest_ind, mannwhitneyu
from pitest_log_parser import mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'
seed_num = len(seed_list)
warnings.filterwarnings('ignore', category=RuntimeWarning, message='Precision loss occurred in moment calculation.')


def get_info(project, seed):
    path = f'{parsed_path}/{choice}/{project}_{seed}'
    with open(f'{path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict


def to_be_calculable(runtime_list):
    output_list = []
    for runtime in runtime_list:
        if runtime == TIMED_OUT:
            output_list.append(6000)
        else:
            output_list.append(runtime)
    return output_list


for project in project_list:
    seed_mutants_dict = {}
    for seed in seed_list:
        mutantId_mutantTuple_dict, mutantId_runtimeList_dict = get_info(project, seed)
        with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as file:
            non_flaky_list = json.load(file)
        seed_mutants_dict[seed] = {}
        for mutant_id in non_flaky_list:
            mutant_pref = tuple(mutantId_mutantTuple_dict[mutant_id][:5])
            seed_mutants_dict[seed][mutant_pref] = to_be_calculable(mutantId_runtimeList_dict[mutant_id])
    for i in range(seed_num):
        i_seed = seed_list[i]
        i_mutants_dict = seed_mutants_dict[i_seed]
        j = i + 1
        while j < seed_num:
            df = pd.DataFrame(None, columns=['mutant', 'T-test', 'U-test'])
            j_seed = seed_list[j]
            j_mutants_dict = seed_mutants_dict[j_seed]
            significant_number = 0
            for pref, i_runtime_list in i_mutants_dict.items():
                j_runtime_list = j_mutants_dict[pref]
                t_stat, t_p_value = ttest_ind(i_runtime_list, j_runtime_list)
                u_stat, u_p_value = mannwhitneyu(i_runtime_list, j_runtime_list)
                if t_p_value < 0.05 or u_p_value < 0.05:
                    significant_number += 1
                df.loc[len(df.index)] = [pref, t_p_value, u_p_value]
            # df.to_csv(f'{analyzed_path}/{choice}/significance_detection/non-flaky/{project}_{i_seed}_{j_seed}.csv', sep=',', header=True, index=False)
            j += 1
            print(f'{project} with ({i_seed}, {j_seed}): {significant_number}/{len(non_flaky_list)}')

## 3. Get total running time except flaky mutants.

In [5]:
import pandas as pd
import json
from scipy.stats import ttest_ind, mannwhitneyu
from pitest_log_parser import mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'
seed_num = len(seed_list)

def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict


def add_list(array1, array2):
    for i in range(round_number):
        if array2[i] == TIMED_OUT:
            array1[i] += 6000
        else:
            array1[i] += int(array2[i])


up_dir = f'{analyzed_path}/{choice}/total_running_time/non-flaky'
significant_df = pd.DataFrame(None, columns=['project', 'seed-pair', 'T-test', 'U-test'])
columns = ['seed'] + [f'round-{i}' for i in range(round_number)]
for project in project_list:
    df = pd.DataFrame(None, columns=columns)
    seed_totalRuntimeList_dict = {}
    for seed in seed_list:
        with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as file:
            non_flaky_list = json.load(file)
        runtime_list = [0 for _ in range(round_number)]
        mutantId_mutantTuple_dict, mutantId_runtimeList_dict = get_info(f'{parsed_path}/{choice}/{project}_{seed}')
        for mutant_id in non_flaky_list:
            add_list(runtime_list, mutantId_runtimeList_dict[mutant_id])
        seed_totalRuntimeList_dict[seed] = runtime_list
        df.loc[len(df.index)] = [seed] + runtime_list
    for i in range(seed_num):
        i_seed = seed_list[i]
        i_array = seed_totalRuntimeList_dict[i_seed]
        for j in range(seed_num):
            if j > i:
                j_seed = seed_list[j]
                j_array = seed_totalRuntimeList_dict[j_seed]
                t_stat, t_p_value = ttest_ind(i_array, j_array)
                u_stat, u_p_value = mannwhitneyu(i_array, j_array)
                significant_df.loc[len(significant_df.index)] = [project, (i_seed, j_seed), t_p_value, u_p_value]
    df.to_csv(f'{up_dir}/{project}.csv', sep=',', header=True, index=False)
significant_df.to_csv(f'{up_dir}/significant_results.csv', sep=',', header=True, index=False)

In [ ]:
# with fastest
import pandas as pd
import numpy as np
import json
from scipy.stats import ttest_ind, mannwhitneyu
from pitest_log_parser import mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'
seed_num = len(seed_list)

def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict


def add_list(array1, array2):
    for i in range(round_number):
        if array2[i] == TIMED_OUT:
            array1[i] += 6000
        else:
            array1[i] += int(array2[i])


up_dir = f'{analyzed_path}/{choice}/total_running_time/non-flaky'
significant_df = pd.DataFrame(None, columns=['project', 'seed', 'T-test', 'U-test', 'reduction_percentage'])
columns = ['seed'] + [f'round-{i}' for i in range(round_number)]
for project in project_list:
    seed_totalRuntimeList_dict = {}
    for seed in seed_list:
        with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as file:
            non_flaky_list = json.load(file)
        runtime_list = [0 for _ in range(round_number)]
        mutantId_mutantTuple_dict, mutantId_runtimeList_dict = get_info(f'{parsed_path}/{choice}/{project}_{seed}')
        for mutant_id in non_flaky_list:
            add_list(runtime_list, mutantId_runtimeList_dict[mutant_id])
        seed_totalRuntimeList_dict[seed] = runtime_list
    # fastest one
    _, mutantId_runtimeList_dict = get_info(f'{parsed_path}/{choice}/{project}_fastest')
    fastest_array = [0 for _ in range(round_number)]
    for _, runtime_list in mutantId_runtimeList_dict.items():
        add_list(fastest_array, runtime_list)
    for seed in seed_list:
        array = seed_totalRuntimeList_dict[seed]
        t_stat, t_p_value = ttest_ind(array, fastest_array)
        u_stat, u_p_value = mannwhitneyu(array, fastest_array)
        avg_rate = np.mean(fastest_array) / np.mean(array)
        significant_df.loc[len(significant_df.index)] = [project, seed, f'{t_p_value:.3f}',
                                                         f'{u_p_value:.3f}', f'{avg_rate:.3f}']
    print(f'{project} with fastest:')
    print(fastest_array)
significant_df.to_csv(f'{up_dir}/significant_results_VS_fastest.csv', sep=',', header=True, index=False)

## 4. Calculate proportions

In [ ]:
import pandas as pd
import random
import json
from pitest_log_parser import mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'
seed_num = len(seed_list)

def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_testsInOrder_dict, mutantId_runtimeList_dict


for project in project_list:
    total_flaky_tests = set()
    for seed in seed_list:
        flaky_chaotic_order_mutant_list = []
        total_num = 0
        non_flaky_num = 0
        flaky_chaotic_order_num = 0
        uniqueId_mutantIds_dict = {}
        mutantId_mutantTuple_dict, mutantId_testsInOrder_dict, mutantId_runtimeList_dict = get_info(f'parsed_data/{choice}/{project}_{seed}')
        for mutant_id, mutant_tuple in mutantId_mutantTuple_dict.items():
            unique_id = tuple(mutant_tuple[:5])
            test_list = mutantId_testsInOrder_dict[mutant_id]
            if unique_id not in uniqueId_mutantIds_dict:
                uniqueId_mutantIds_dict[unique_id] = set()
            uniqueId_mutantIds_dict[unique_id].add(mutant_id)
        total_num = len(uniqueId_mutantIds_dict)
        for unique_id, mutant_ids in uniqueId_mutantIds_dict.items():
            if len(mutant_ids) == 1:
                non_flaky_num += 1
            else:
                test_intersection_set = None
                for mutant_id in mutant_ids:
                    per_test_set = set(mutantId_testsInOrder_dict[mutant_id])
                    if test_intersection_set is None:
                        test_intersection_set = per_test_set
                    else:
                        test_intersection_set &= per_test_set
                is_same = True
                flaky_tests = set()
                for mutant_id in mutant_ids:
                    per_test_set = set(mutantId_testsInOrder_dict[mutant_id])
                    if len(per_test_set) == len(test_intersection_set):
                        continue
                    flaky_tests |= (per_test_set - test_intersection_set)
                if len(flaky_tests) == 0:
                    flaky_chaotic_order_num += 1
                    flaky_chaotic_order_mutant_list.append(random.choice(list(mutant_ids)))
                total_flaky_tests |= flaky_tests
        # with open(f'analyzed_data/{choice}/mutant_list/chaotic_order/{project}_{seed}.json', 'w') as file:
        #     json.dump(flaky_chaotic_order_mutant_list, file, indent=4)
    
        print(f'{project} with {seed}:')
        print(f'Total mutants: {total_num}')
        print(f'Non-flaky mutants: {non_flaky_num} ({non_flaky_num / total_num:.4f})')
        print(f'Flaky mutants due to chaotic test list: {flaky_chaotic_order_num} ({flaky_chaotic_order_num / total_num:.4f})')

## 5. Create guiding files

In [ ]:
import random
import json
from pitest_log_parser import project_junitVersion_dict, mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'

def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_testsInOrder_dict


def mutant_to_json(mutant, test_list, junit_version):
    def test_sort(test):
        return (test['name'])
    
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index-1],
                'name': test
            })
    if random_test:
        random.shuffle(test_order)
    else:
        test_order.sort(key=test_sort)

    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'filename': mutant[5],
        'block': f'[{mutant[6]}]',
        'lineNumber': int(mutant[7]),
        'description': mutant[8],
        'testsInOrder': test_order
    }


def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        mutant['id']['location']['method'],
        mutant['id']['location']['methodDesc'],
        mutant['id']['indexes'],
        mutant['id']['mutator']
    )


parent_path = f'analyzed_data/{choice}/mutant_list'
for project in project_list:
    junit_version = project_junitVersion_dict[project]
    # get the mutant intersection
    id_set = set()
    for seed in seed_list:
        mutantId_mutantTuple_dict, _ = get_info(f'parsed_data/{choice}/{project}_{seed}')
        with open(f'{parent_path}/chaotic_order/{project}_{seed}.json', 'r') as file:
            chaotic_order_list = json.load(file)
        per_id_set = set()
        for mutant_id in chaotic_order_list:
            per_id_set.add(tuple(mutantId_mutantTuple_dict[mutant_id][:5]))
        if len(id_set) == 0:
            id_set |= per_id_set
        else:
            id_set &= per_id_set
    # create guiding mutant list
    for seed in seed_list:
        with open(f'{parent_path}/non-flaky/{project}_{seed}.json', 'r') as file:
            non_flaky_list = json.load(file)
        mutantId_mutantTuple_dict, mutantId_testsInOrder_dict = get_info(f'parsed_data/{choice}/{project}_{seed}')
        output_list = []
        id_visited_dict = {item: True for item in id_set}
        for mutant_id, mutant_tuple in mutantId_mutantTuple_dict.items():
            id_tuple = tuple(mutant_tuple[:5])
            if mutant_id in non_flaky_list or (id_tuple in id_set and id_visited_dict[id_tuple]):
                if id_tuple in id_set:
                    id_visited_dict[id_tuple] = False
                output_list.append(mutant_to_json(mutant=mutant_tuple,
                                                  test_list=mutantId_testsInOrder_dict[mutant_id],
                                                  junit_version=junit_version))
        if random_mutant:
            random.shuffle(output_list)
        else:
            output_list.sort(key=mutant_sort)
        with open(f'analyzed_data/{choice}/guiding_files/{project}_{seed}.json', 'w') as f:
            f.write(json.dumps(output_list, indent=4))

## 6. Choose fastest test order to guide

In [25]:
import random
import json
from pitest_log_parser import project_junitVersion_dict, mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'

def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict, mutantId_testsInOrder_dict


def to_be_calculable(runtime):
    if runtime == TIMED_OUT:
        return 6000
    else:
        return int(runtime)
    

def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        mutant['id']['location']['method'],
        mutant['id']['location']['methodDesc'],
        mutant['id']['indexes'],
        mutant['id']['mutator']
    )
    

def mutant_to_json(mutant, test_list, junit_version):
    def test_sort(test):
        return (test['name'])
    
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index-1],
                'name': test
            })
    if random_test:
        random.shuffle(test_order)
    else:
        test_order.sort(key=test_sort)
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'filename': mutant[5],
        'block': f'[{mutant[6]}]',
        'lineNumber': int(mutant[7]),
        'description': mutant[8],
        'testsInOrder': test_order
    }


for project in project_list:
    junit_version = project_junitVersion_dict[project]
    mutation_runtime_dict = {}
    mutation_tests_dict = {}
    is_visited = False
    for seed in seed_list:
        with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as file:
            non_flaky_list = json.load(file)
        mutantId_mutantTuple_dict, mutantId_runtimeList_dict, mutantId_testsInOrder_dict = get_info(f'{parsed_path}/{choice}/{project}_{seed}')
        for mutant_id in non_flaky_list:
            mutant = tuple(mutantId_mutantTuple_dict[mutant_id])
            new_runtime = sum([to_be_calculable(runtime) for runtime in mutantId_runtimeList_dict[mutant_id]])
            if not is_visited:
                mutation_tests_dict[mutant] = mutantId_testsInOrder_dict[mutant_id]
                mutation_runtime_dict[mutant] = new_runtime
            else:
                if new_runtime < mutation_runtime_dict[mutant]:
                    mutation_tests_dict[mutant] = mutantId_testsInOrder_dict[mutant_id]
                    mutation_runtime_dict[mutant] = new_runtime
        is_visited = True
    output_list = []
    for mutant, testsInOrder in mutation_tests_dict.items():
        output_list.append(mutant_to_json(
            mutant=mutant,
            test_list=testsInOrder,
            junit_version=junit_version
        ))
    output_list.sort(key=mutant_sort)
    with open(f'{analyzed_path}/{choice}/guiding_files/{project}.json', 'w') as f:
        f.write(json.dumps(output_list, indent=4))

## 7. About mutation xml

In [59]:
import pandas as pd
import json
from pitest_log_parser import project_junitVersion_dict, mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'
seed_number = len(seed_list)

def mutation_filter(mutations, non_flaky_list, mutantId_mutantTuple_dict):
    non_flaky_mutations = []
    for idx in non_flaky_list:
        mutuple = mutantId_mutantTuple_dict[idx]
        indexes = None
        if ',' in mutuple[3]:
            indexes = mutuple[3].split(', ')
        else:
            indexes = [mutuple[3]]
        non_flaky_mutations.append({
            'mutatedClass': mutuple[0],
            'mutatedMethod': mutuple[1],
            'methodDescription': mutuple[2],
            'indexes': indexes,
            'mutator': mutuple[4]
            })
    filtered_mutations = []
    filtered_indexes = []
    keys_to_extract = ['mutatedClass', 'mutatedMethod', 'methodDescription', 'indexes', 'mutator']
    for i, mut in enumerate(mutations):
        if {k: mut[k] for k in keys_to_extract} in non_flaky_mutations:
            filtered_mutations.append(mut)
            filtered_indexes.append(i)
    return filtered_mutations, filtered_indexes


def is_different(tests0, tests1):
    if tests0 is None and tests1 is None:
        return False
    if tests0 is None or tests1 is None:
        return True
    if set(tests0) == set(tests1):
        return False
    return True


def test_recorder(test_statuses_dict, i, tests, status):
    if tests is None:
        return
    for t in tests:
        if not t in test_statuses_dict:
            test_statuses_dict[t] = ['' for _ in range(seed_number)]
        if test_statuses_dict[t][i] == '' or test_statuses_dict[t][i] == status:
            test_statuses_dict[t][i] = status
        else:
            test_statuses_dict[t][i] = 'killed/survived'


parent_path = f'controlled_parsed_data/{choice}'
for project in project_list:
    muid_isConsistent_dict = {}
    muid_killedTests_dict = {}
    muid_succeedingTests_dict = {}
    muid_indexMatrix_dict = {}
    is_visited = False
    for i, seed in enumerate(seed_list):
        with open(f'{parent_path}/{project}_{seed}/mutations_xml.json', 'r') as file:
            mutation_list = json.load(file)
        with open(f'{parent_path}/{project}_{seed}/mutantId_mutantTuple.json', 'r') as file:
            mutantId_mutantTuple_dict = json.load(file)
        with open(f'controlled_analyzed_data/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as file:
            non_flaky_list = json.load(file)
        filtered_mutations, filtered_indexes = mutation_filter(mutation_list,
                                                               non_flaky_list,
                                                               mutantId_mutantTuple_dict)
        if not is_visited:
            for mutation in filtered_mutations:
                muid = (mutation['mutatedClass'], mutation['mutatedMethod'], mutation['methodDescription'])
                muid += (str(mutation['indexes']), mutation['mutator'])
                muid_isConsistent_dict[muid] = True
                muid_indexMatrix_dict[muid] = [[] for _ in range(seed_number)]
                muid_killedTests_dict[muid] = mutation['killingTests']
                muid_succeedingTests_dict[muid] = mutation['succeedingTests']
         
        # the same order has different statuses.
        for j, mutation in enumerate(filtered_mutations):
            muid = (mutation['mutatedClass'], mutation['mutatedMethod'], mutation['methodDescription'])
            muid += (str(mutation['indexes']), mutation['mutator'])
            if len(muid_indexMatrix_dict[muid][i]) >= 1:
                k = muid_indexMatrix_dict[muid][i][0]
                if is_different(filtered_mutations[k]['killingTests'], mutation['killingTests']) or is_different(filtered_mutations[k]['succeedingTests'], mutation['succeedingTests']):
                    muid_isConsistent_dict[muid] = False
            muid_indexMatrix_dict[muid][i] += [filtered_indexes[j]]
        
        # the different order has different statuses.
        if is_visited:
            for j, mutation in enumerate(filtered_mutations):
                muid = (mutation['mutatedClass'], mutation['mutatedMethod'], mutation['methodDescription'])
                muid += (str(mutation['indexes']), mutation['mutator'])
                if not muid_isConsistent_dict[muid]:
                    continue
                killing_tests = mutation['killingTests']
                succeeding_tests = mutation['succeedingTests']
                if is_different(muid_killedTests_dict[muid], killing_tests) or is_different(muid_succeedingTests_dict[muid], succeeding_tests):
                    muid_isConsistent_dict[muid] = False  
        is_visited = True
    cnt = 0
    df = pd.DataFrame(None, columns=['mutated_class', 'mutated_method', 'method_description', 'indexes', 'mutator',
                                     'test'] + [f'seed_{s}' for s in seed_list])
    for muid, is_consistent in muid_isConsistent_dict.items():
        if is_consistent:
            continue
        cnt += 1
        test_statuses_dict = {}
        for i, indexes in enumerate(muid_indexMatrix_dict[muid]):
            with open(f'{parent_path}/{project}_{seed_list[i]}/mutations_xml.json', 'r') as file:
                mutation_list = json.load(file)
            for index in indexes:
                mutation = mutation_list[index]
                test_recorder(test_statuses_dict, i, mutation['killingTests'], 'killed')
                test_recorder(test_statuses_dict, i, mutation['succeedingTests'], 'survived')
        for i, indexes in enumerate(muid_indexMatrix_dict[muid]):
            with open(f'{parent_path}/{project}_{seed_list[i]}/mutations_xml.json', 'r') as file:
                mutation_list = json.load(file)    
            for index in indexes:
                mutation = mutation_list[index]
                if mutation['status'] in [TIMED_OUT, 'MEMORY_ERROR']:
                    for t, statuses in test_statuses_dict.items():
                        if test_statuses_dict[t][i] == '':
                            test_statuses_dict[t][i] = mutation['status']
        for test, statuses in test_statuses_dict.items():
            if len(set(statuses)) == 1:
                continue
            df.loc[len(df.index)] = [mutation['mutatedClass'], mutation['mutatedMethod'], mutation['methodDescription'],
                                     mutation['indexes'], mutation['mutator'], test] + statuses
    df.to_csv(f'controlled_analyzed_data/{choice}/xml_results/{project}.csv', sep=',', header=True, index=False)
    print(f'{project}: {cnt}/{len(muid_isConsistent_dict)}')

commons-codec: 14/3551
delight-nashorn-sandbox: 28/225
empire-db: 14/2232
jimfs: 20/1520
